#  Word2vec

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://radimrehurek.com/gensim/models/word2vec.html
* https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html
* https://pytorch.org/text/stable/vocab.html
* https://github.com/OlgaChernytska/word2vec-pytorch
* https://www.baeldung.com/cs/nlps-word2vec-negative-sampling
* https://towardsdatascience.com/implementing-word2vec-in-pytorch-from-the-ground-up-c7fe5bf99889

## Задачи для совместного разбора

1\. Рассмотрите основные шаги подготовки данных для обучения skip-gram модели

In [2]:
text = "Спящий котик мило моргает своими яркими глазками"

In [4]:
from torchtext.vocab import build_vocab_from_iterator

In [7]:
corpus = [
    text.lower().split()
]
corpus

[['спящий', 'котик', 'мило', 'моргает', 'своими', 'яркими', 'глазками']]

In [8]:
vocab = build_vocab_from_iterator(corpus, )

corpus_i = [
    vocab.lookup_indices(t)
    for t in corpus
]
corpus_i

[[5, 1, 2, 3, 4, 6, 0]]

In [11]:
example = corpus_i[0]
inputs, outputs = [], []

for idx, word in enumerate(example):
  if idx == 0 or idx == len(example) - 1:
    continue
  inputs.append(word)
  outputs.append(example[idx-1])

  inputs.append(word)
  outputs.append(example[idx+1])

In [28]:
import pandas as pd

pd.DataFrame({"x": inputs, "y": outputs})

,x,y
0,1,5
1,1,2
2,2,1
3,2,3
4,3,2
5,3,4
6,4,3
7,4,6
8,6,4
9,6,0


In [14]:
import torch as th

inputs = th.tensor(inputs, dtype=th.long)
outputs = th.tensor(outputs, dtype=th.long)

2\. Рассмотрите основные шаги по настройке skip-gram модели

In [16]:
import torch.nn as nn

embeddings = nn.Embedding(num_embeddings=len(vocab), embedding_dim=300)

In [18]:
len(inputs), embeddings(inputs).shape

(10, torch.Size([10, 300]))

In [19]:
len(vocab)

7

In [20]:
fc = nn.Linear(in_features=300, out_features=len(vocab))

In [ ]:
x_e = embeddings(inputs)
out = fc(x_e)
out

In [24]:
outputs

tensor([5, 2, 1, 3, 2, 4, 3, 6, 4, 0])

In [27]:
model = nn.Sequential(
    nn.Embedding(num_embeddings=len(vocab), embedding_dim=300),
    nn.Linear(in_features=300, out_features=len(vocab))
)
criterion = nn.CrossEntropyLoss()

out = model(inputs)
loss = criterion(out, outputs)
loss

tensor(2.0006, grad_fn=<NllLossBackward0>)

In [33]:
embeddings = nn.Embedding(
    num_embeddings=len(vocab),
    embedding_dim=16,
    max_norm=1
)

inputs_e = embeddings(inputs)
outputs_e = embeddings(outputs)

In [34]:
outputs[0]

tensor(5)

In [38]:
inputs_e[0] @ outputs_e[0]

tensor(0.0742, grad_fn=<DotBackward0>)

In [39]:
outputs[0], inputs[0]

(tensor(5), tensor(1))

In [41]:
(inputs_e @ outputs_e.T).shape

torch.Size([10, 10])

In [47]:
y_pred = th.tensor([
    [1, 0.0742]
]) # логиты
# y_pred.softmax(dim=1) # вероятности
# y_pred.sigmoid()
y = [0]

In [ ]:
o = inputs_e.view(-1, 1, 16).bmm(outputs_e.view(-1, 16, 1))
o

## Задачи для самостоятельного решения

In [8]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


<p class="task" id="1"></p>

1\. Загрузите тексты новостей из файла `news_500.csv`. Удалите из текстов все знаки препинания и символы не из русского алфавита, приведите все слова к нижнему регистру и удалите стоп-слова. Разбейте текст каждой новости на фрагменты по 3 предложения и сохраните в виде списка строк. Выведите на экран длину полученного списка.

- [x] Проверено на семинаре

In [53]:
data = pd.read_csv('data/news_500.csv')
data = data.sample(2)

In [54]:
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\sа-яА-Я]', '', text)
    words = cleaned_text.lower().split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [55]:
fragmented_text = []
for news_text in data['text']:
    sentences = sent_tokenize(news_text)
    fragments = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
    fragmented_text.append(fragments)


fragmented_text

[['Сейчас на территории нашей страны действует около 8 000 обменных пунктов, две трети из которых расположены в Москве. Как отметили в ГУБЭП, на протяжении последних лет "обменники" стали предметом особого внимания преступников, которые используют их для легализации преступно нажитых средств и ухода от налогов. Сотрудники подразделений УБЭП совместно с представителями Банка России провели проверку деятельности обменных пунктов и пришли к выводу, что "карательными мерами" изменить ситуацию нельзя.',
  'Необходимо вносить изменения в законодательство. В МВД пока предпочитают подробно не комментировать те меры, которые будут предложены этим ведомством руководству Центробанка. Однако, как полагают многие аналитики, часть этих предложений будет касаться статуса справок об обмене валюты, которые выдаются гражданам при обмене денег.',
  'На данный момент справка о купле-продаже валюты является разрешением на ее вывоз, ее также обязаны принимать как документ, подтверждающий легальное происхожд

<p class="task" id="2"></p>

2\. Настройте модель Word2Vec из пакета `gensim`. Для валидации выведите на экран информацию о ближайших словах для нескольких случайно выбранных токенов из обучающей выборки.

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

3. Опишите класс `W2VDataset`, который реализует в себе логику получения контекстного окна для обучения skip-gram модели. При создании словаря игнорируйте токены, которые встретились меньше 20 раз. Продемонстрируйте пример работы.

![image.png](attachment:image.png)

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Реализуйте и настройте skip-gram модель. Перед началом обучения выберите случайным образом несколько слов из датасета и для каждого из них выведите на экран 3 ближайших слова в смысле косинусной близости между эмбеддингами. В процессе настройки для валидации периодически выводите на экран информацию о ближайших словах для этих слов. Выведите на экран график значения функции потерь в зависимости от номера эпохи.  

![image.png](attachment:image.png)

- [ ] Проверено на семинаре

<p class="task" id="5"></p>

5\. Реализуйте класс `NegativeSampler`, который позволяет сгенерировать набор отрицательных примеров. Для генерации отрицательных примеров выбирайте токены пропорционально величине $C(w)^{\frac{3}{4}}$, где $C(w)$ - частота токена в корпусе.


- [ ] Проверено на семинаре

<p class="task" id="6"></p>

6\. Реализуйте и настройте skip-gram модель с использованием negative sampling. Перед началом обучения выберите случайным образом несколько слов из датасета и для каждого из них выведите на экран 3 ближайших слова в смысле косинусной близости между эмбеддингами. В процессе настройки для валидации периодически выводите на экран информацию о ближайших словах для этих слов. Выведите на экран график значения функции потерь в зависимости от номера эпохи.  

- [ ] Проверено на семинаре

## Обратная связь
- [ ] Хочу получить обратную связь по решению